# ML Course 3: Classification:  Week 1 Assignment 1
## Predicting sentiment from product reviews
## Pandas sklearn


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started! see also https://github.com/smarthi/UnivOfWashington-Machine-Learning/blob/master/Classification/week1/module-2-linear-classifier-assignment-blank.ipynb

see also http://logisticregressionanalysis.com/909-understanding-logistic-regression-outputpart-4-making-predictions/

also http://www.ats.ucla.edu/stat/stata/webbooks/logistic/chapter1/statalog1.htm
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [313]:
from __future__ import division
import pandas as pd
import numpy as np
import math
import string

# Data preperation

**1.** Load the dataset consisting of baby product reviews on Amazon.com. Store the data in a data frame products.

In [314]:
#products = sf.SFrame('../../course-1/Week3/amazon_baby.csv')
dtype_dict = {'name':str, 'review':str, 'rating':int}
products = pd.read_csv('../../course-1/Week3/amazon_baby.csv', dtype = dtype_dict)
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
name      183213 non-null object
review    182702 non-null object
rating    183531 non-null int64
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


Now, let us see a preview of what the dataset looks like.

In [315]:
print "Features are ",products.columns
print "Number of features = %d" %len(products.columns)
(num_rows,num_cols) = products.shape
print "Number of data items = %d" %products.size
print products.ndim
print products.index
print "Number of samples/rows = %d and num of features/cols =  %d" %products.shape
products.head(2)

Features are  Index([u'name', u'review', u'rating'], dtype='object')
Number of features = 3
Number of data items = 550593
2
RangeIndex(start=0, stop=183531, step=1)
Number of samples/rows = 183531 and num of features/cols =  3


,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [316]:
# products.describe()
print products.iloc[269] #iloc: Purely integer-location based indexing for selection by position
print products.loc[269]  #loc:  Purely label based indexing for selection by label
products[269:270]          #slicing also works

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object
name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object


,name,review,rating
269,The First Years Massaging Action Teether,A favorite in our house!,5


**2.** Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality, so that words "cake." and "cake!" are counted as the same word.
2. Apply this function to every element in the review column of products, and save the result to a new column review_clean. (Transform the reviews into word-counts. - not doing this here)

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

IMPORTANT. Make sure to fill n/a values in the review column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [317]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [318]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

#review_without_puctuation = products['review'].apply(remove_punctuation)
#products['word_count'] = sf.text_analytics.count_words(review_without_puctuation)

In [319]:
#print products.dtypes
#products['review'] = products['review'].astype(str) 

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [320]:
# print products[269]['review']      # KeyError 269
print products.iloc[269]['review']   # Use iloc for positional indexing
print products.loc[269]['review']    # Use loc for label-based indexing

products[269:270]['review']         #normal python slicing also works  
#products = products.fillna({'review':''})  # fill in N/A's in the review column

A favorite in our house!
A favorite in our house!


269    A favorite in our house!
Name: review, dtype: object

## Extract sentiments

**3.** We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment. 

In [321]:
print len(products)
products = products[ products['rating'] != 3 ]
print len(products)

183531
166752


**4.** Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the rating column. In SFrame, you would use apply():

In [322]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
len(products) == len ( products[products['sentiment'] == -1]) + \
                len ( products[products['sentiment'] == 1])

True

In [323]:
products[ products['rating'] > 3].head(2)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1


In [324]:
products[ products['rating'] < 3].head(2)

,name,review,rating,review_clean,sentiment
21,Nature\'s Lullabies Second Year Sticker Calendar,I only purchased a second-year calendar for my...,2,I only purchased a secondyear calendar for my ...,-1
41,"SoftPlay Giggle Jiggle Funbook, Happy Bear",This bear is absolutely adorable and I would g...,2,This bear is absolutely adorable and I would g...,-1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1). We can also see that all product reviews got a sentiment (+1 or -1) cause the sum of -1 and +1 sentiments add up to number of product reviews we have.

## Split data into training and test sets

**5.** Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result. 

We are not using SFrame, so download the list of indices for the training and test sets: module-2-assignment-train-idx.json, module-2-assignment-test-idx.json. IMPORTANT: If you are using a programming language with 1-based indexing (e.g. R, Matlab), make sure to increment all indices by 1.

we will need a way to pass a list of indices/integers - [see](http://pandas.pydata.org/pandas-docs/stable/indexing.html) and [this](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes)

*    Use loc for label-based indexing
*    Use iloc for positional indexing - purely integer based indexing. df.iloc[]

In [325]:
#train_data, test_data = products.random_split(.8, seed=1)
#print len(train_data)  #133416
#print len(test_data)   #33336

In [326]:
import json
with open('module2_assignment_train_idx.json', 'r') as file1: 
    train_idx = json.load(file1)   #reads entire file in one go 

with open('module2_assignment_test_idx.json', 'r') as file2: 
    test_idx = json.load(file2)    #reads entire file in one go 
 
train_data = products.iloc[train_idx, :]
test_data =  products.iloc[test_idx,  :]
print len(train_data)  #133416
print len(test_data)   #33336

133416
33336


### Build the word count vector for each review

**6.** We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

*    Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
*    Compute the occurrences of the words in each review and collect them into a row vector.
*    Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
*    Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [327]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words

# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])

# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. 

Instructions for graphlab --> This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**7.** Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

**Note:** This line may take 1-2 minutes.

In [328]:
print train_matrix.shape   
# as many samples as that in train set, and 12712 columns/features!!!

(133416, 121712)


In [329]:
from sklearn.linear_model import LogisticRegression
# instantiate a logistic regression model, and fit with X and y
# X -> sparse matrix where each row is the word count vector for the corresponding review
model = LogisticRegression()
X = train_matrix 
y = train_data['sentiment']
sentiment_model = model.fit(X, y)

# check the accuracy on the training set
sentiment_model.score(X, y)                                                  

0.96851951789890267

In [330]:
sentiment_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

**8.** There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients. 

In [331]:
print sentiment_model.coef_
print "\nNon zero coefficients (including intercept) = " , \
np.count_nonzero(sentiment_model.coef_) + np.count_nonzero(sentiment_model.intercept_)
#len(model_all.sparse_coef_)
print "(sentiment_model.coef_).shape[0]", (sentiment_model.coef_).shape[0]
#print sentiment_model.coef_.getnnz()
print "sentiment_model.coef_.shape:", sentiment_model.coef_.shape

[[ -1.24016858e+00   2.97817122e-05   2.75239185e-02 ...,   1.26002631e-02
    2.71201093e-03  -4.16780197e-05]]

Non zero coefficients (including intercept) =  121713
(sentiment_model.coef_).shape[0] 1
sentiment_model.coef_.shape: (1, 121712)


In [332]:
# examine the coefficients - can't do sparse matrix X has no column names
# pd.DataFrame(zip(X.columns, np.transpose(sentiment_model.coef_))).head()

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [333]:
#build list of 1's, 1 if weight +ve, then sum the 1s'!!
np1 = sum(1 for x in sentiment_model.coef_[0] if x > 0)    

#build list - then count num of elements
np2 = len([x for x in sentiment_model.coef_[0] if x > 0])  
print np1
print np2

#compare this with traditional way of iterating through list, about 7 lines code...
count_pos = 0
count_neg = 0
for i in range(len(sentiment_model.coef_[0])):
    if sentiment_model.coef_[0][i] >= 0:
        count_pos=count_pos+1
    else:
        count_neg=count_neg+1

print count_pos + count_neg
print "Number of positive weights: %s " % count_pos
print "Number of negative weights: %s " % count_neg
#print "Total: %s " % (num_positive_weights + num_negative_weights)

86906
86906
121712
Number of positive weights: 86906 
Number of negative weights: 34806 


**Quiz question:** How many weights are >= 0?  68419 (according to graphlab)

## Making predictions with logistic regression
**9.** Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 data points/examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data. Python's regular slicing syntax wont work - sample_test_data['review'][0], cause the indexes are those from test_data!!

In [334]:
sample_test_data = test_data[10:13]  #samples/rows 10,11,12
print sample_test_data['rating']   # Please NB row indices - use that with iloc!!
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: int64


,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

Good time to review Different Choices for Indexing for dataframes, [see] (http://pandas.pydata.org/pandas-docs/stable/indexing.html)

**.loc is primarily label based**, but may also be used with a boolean array. .loc will raise KeyError when the items are not found. Allowed inputs are:

*    A single label, e.g. 5 or 'a', (note that **in this context 5 is interpreted as a label of the index. This use is not an integer position along the index)**
*    A list or array of labels ['a', 'b', 'c']
*    A slice object with labels 'a':'f', (note that contrary to usual python slices, both the start and the stop are included!)
*    A boolean array e.g products[ products['rating'] != 3 ]
*    A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above)

**.iloc is primarily integer position based (from 0 to length-1 of the axis)**, but may also be used with a boolean array. .iloc will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with python/numpy slice semantics). Allowed inputs are:

*    An integer e.g. 5, its interpreted as an integer position along the index
*    A list or array of integers [4, 3, 0] e.g train_data = products.iloc[train_idx, :], where train_idx is a list of integer indices.
*    A slice object with ints e.g 1:7  --> test_data[10:13]
*    A boolean array e.g products[products['sentiment'] == -1]
*    A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above)

In [346]:
# sample_test_data[59]['review']      #KeyError: 59
# sample_test_data[0]['review']       #KeyError:  0

print sample_test_data.loc[59]['review'] 
# sample_test_data.iloc[59]['review']
# IndexError: single positional indexer is out-of-bounds

# Use iloc for positional indexing
print sample_test_data.iloc[0]['review']

# print sample_test_data.loc[0]['review'] 
# KeyError: 'the label [0] is not in the [index]'

print sample_test_data['review'][59]
print sample_test_data['review'][59:60]

# see http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/

Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Series([], Name: review, dtype: object)


That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [347]:
sample_test_data['review'][71]

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

**10.** We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**. Note that scores is just a dot product of weights and features as in regression!! - so predicted values.

Hint: need to convert sample_test_data into the sparse matrix format first.

In [348]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60224818  -3.17079879 -10.42488847]


In [349]:
# margin classfier --> margin=score, 
# score >= 0 on one side of margin, score<0 on other side of margin or decision boundary
#scores = sentiment_model.predict(sample_test_data, output_type='margin')
#print scores   
#print scores.dtype

### Predicting sentiment

**11.** These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions: 

Again there are many ways to skin the goat!!! Given scores is just a list of values, we can either use an in place lambda function of list comprehension to do this in one line, show off both ways below. Traditioanlly we would iterate through the list and pick up item/value one by one, inspect if its neg or pos and the store/print corresponding -1 or +1. In both cases a list of outcomes is produced.


In [350]:
#'numpy.ndarray' object has no attribute 'apply'
#predictions = scores.apply(lambda x : +1 if x > 0 else -1)  #lambda in place function
#print predictions                                           #scores is SArray.dtype: float

In [351]:
#list comprehension  - optimized iterator through values in list, for each item do test
predictions = [+1 if x > 0 else -1 for x in scores]    
print predictions

[1, -1, -1]


In [352]:
def predict_class(model, data):
    scores = list(model.decision_function(data))
    score_class = [+1 if x > 0 else -1 for x in scores]
    return (score_class)  #if no enclose return, prints None!!

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [353]:
print "My class predictions:"
print predict_class(sentiment_model, sample_test_matrix )
print "Class predictions according to  scikit-learn:" 
print sentiment_model.predict(sample_test_matrix)

My class predictions:
[1, -1, -1]
Class predictions according to  scikit-learn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

### Probability predictions

**12.** Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{Score})}.
$$ $$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**. 

Note that the probability values are for a prediction of positive sentiment (+1) given a specific value for featuere or input x and its weight/coefficient. so $$ P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) $$ 
We can easily find probability of a negative sentiment, just 1 - P(+1) !!

In [354]:
import numpy as np
probabilities = 1.0 * 1/(1 + np.exp(-scores))
print probabilities

[  9.96324003e-01   4.02795251e-02   2.96835318e-05]


**Checkpoint**: Make sure your probability predictions match the ones obtained from  scikit-learn. Note Class predictions using GraphLab Create 'sentiment_model.predict(sample_test_data, output_type='probability')' gives $$ P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) $$ 
i.e probability of +ve review/+ve sentiment P(y=+1) given X and weights. sklearn provides probabilities for both class, 1st colum is probs for -ve class, 1nd col is probs for +ve class.

In [355]:
print "Class predictions according to sklearn:" 
probs = sentiment_model.predict_proba(sample_test_matrix)
print probs

#
#probabilities in 2nd column are for class 1 - +ve reviews, 1st col probs for -ve reviews
print "\nProbability of +ve reviews",probs[:,1] 


Class predictions according to sklearn:
[[  3.67599678e-03   9.96324003e-01]
 [  9.59720475e-01   4.02795251e-02]
 [  9.99970316e-01   2.96835318e-05]]

Probability of +ve reviews [  9.96324003e-01   4.02795251e-02   2.96835318e-05]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? **third** cause it has lowest value to power of -5.

# Find the most positive (and negative) review

**13.** We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.) sklearns predict_proba() outputs a n by 2 matrix, n samples and 1 col each for each of the two classes. In this case we can see from above code segment that the +ve class probabibilities are in 2nd col, and -ve class probs in col 1.

In [356]:
#get 1st 3 predictions using test_matrix to test code
prob_pred = sentiment_model.predict_proba(test_matrix)[0:3, ]
print "Shape of predict_proba()", np.shape(prob_pred)    
#5 predictions - 1 for each sample in test_matrix and prob pred are for both classes
# which column is prediction for +ve class ???
#prob_pred = prob_pred.tolist()   #no need for this, already a 2D numpy array

print prob_pred

prob_pred = prob_pred[:,1]   #assume 2nd col for +ve class, grab that only for sorting
print "2nd col is prob for +ve class", prob_pred

#sorts probabilities ascending order and gives indices of sorted list
order = np.argsort(prob_pred)  
print "\nIndices for probs sorted ascending", order
print "\nActual Probs sorted ascending", prob_pred[order]


#great - now apply to entire data set

prob_pred = sentiment_model.predict_proba(test_matrix)
print "\nShape of predict_proba()", np.shape(prob_pred)    
#33336 predictions - 1 for each sample and prob pred for both classes
print prob_pred

prob_pred = prob_pred[:,1]   #assume 2nd col for +ve class, grab that only for sorting
print "2nd col is prob for +ve class", prob_pred

order=np.argsort(prob_pred) 
print "\nIndices for probs sorted ascending", order
print "\nActual Probs sorted ascending", prob_pred[order][-20:]
#print test_data['review_clean'][order]

Shape of predict_proba() (3, 2)
[[  2.15452332e-01   7.84547668e-01]
 [  7.64698063e-07   9.99999235e-01]
 [  6.67993225e-02   9.33200677e-01]]
2nd col is prob for +ve class [ 0.78454767  0.99999924  0.93320068]

Indices for probs sorted ascending [0 2 1]

Actual Probs sorted ascending [ 0.78454767  0.93320068  0.99999924]

Shape of predict_proba() (33336, 2)
[[  2.15452332e-01   7.84547668e-01]
 [  7.64698063e-07   9.99999235e-01]
 [  6.67993225e-02   9.33200677e-01]
 ..., 
 [  5.15140529e-06   9.99994849e-01]
 [  2.39182548e-06   9.99997608e-01]
 [  1.82340001e-02   9.81766000e-01]]
2nd col is prob for +ve class [ 0.78454767  0.99999924  0.93320068 ...,  0.99999485  0.99999761  0.981766  ]

Indices for probs sorted ascending [ 2931 21700 13939 ..., 18112 20743 15732]

Actual Probs sorted ascending [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]


In [357]:
prob_pred = sentiment_model.predict_proba(test_matrix)
print "\nShape of predict_proba()", np.shape(prob_pred)    

prob_pred = prob_pred[:,1]   #assume 2nd col for +ve class, grab that only for sorting
print "2nd col is prob for +ve class", prob_pred

order = np.argsort(prob_pred)   #sort lowest to highest prob

#now we have list of indices for rows test matrix with highest prob +ve review
top20_indx = order[-20:]  
test_data.iloc[top20_indx,:]


Shape of predict_proba() (33336, 2)
2nd col is prob for +ve class [ 0.78454767  0.99999924  0.93320068 ...,  0.99999485  0.99999761  0.981766  ]


,name,review,rating,review_clean,sentiment
172351,Phil &amp; Teds Navigator Buggy Golden Kiwi Fr...,I\'m pretty happy with this stroller. I use it...,4,Im pretty happy with this stroller I use it wi...,1
182089,Summer Infant Wide View Digital Color Video Mo...,I love this baby monitor. I can compare this ...,5,I love this baby monitor I can compare this o...,1
165593,Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatwa...,For the price this set is unbelievable- and tr...,5,For the price this set is unbelievable and tru...,1
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,1
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It\'s always fun to write a review on those pr...,5,Its always fun to write a review on those prod...,1
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1
137034,Graco Pack \'n Play Element Playard - Flint,My husband and I assembled this Pack n\' Play ...,4,My husband and I assembled this Pack n Play la...,1
180646,Mamas &amp; Papas 2014 Urbo2 Stroller - Black,After much research I purchased an Urbo2. It\'...,4,After much research I purchased an Urbo2 Its e...,1


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]

Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [358]:
#now we have list of indices for rows test matrix with lowest prob +ve review
lowest20_indx = order[0:20]  
test_data.iloc[lowest20_indx,:]

,name,review,rating,review_clean,sentiment
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1
120209,Levana Safe N\'See Digital Video Baby Monitor ...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1
10677,Philips AVENT Newborn Starter Set,"It\'s 3am in the morning and needless to say, ...",1,Its 3am in the morning and needless to say thi...,-1
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1


In [359]:
# or alternatively sort the probability on -ve class lowest to highest
# and last 20 will with those with highest prob for being a -ve review
# Gives same result as reviews with lowest prob being +ve

prob_pred = sentiment_model.predict_proba(test_matrix)
print "\nShape of predict_proba()", np.shape(prob_pred)    

prob_pred = prob_pred[:,0]   #assume 2nd col for +ve class, grab that only for sorting
print "1st col is prob for -ve class", prob_pred

order = np.argsort(prob_pred)   #sort lowest to highest prob

#now we have list of indices for rows test matrix with highest prob -ve review
top20_indx = order[-20:]  



Shape of predict_proba() (33336, 2)
1st col is prob for -ve class [  2.15452332e-01   7.64698063e-07   6.67993225e-02 ...,   5.15140529e-06
   2.39182548e-06   1.82340001e-02]


Now we just need to show that both lists conatin same row indices. An easy (and quick) way to determine if two unordered lists contain the same elements. Python has a built-in datatype for an unordered collection of (hashable) things, called a set. If you convert both lists to sets, the comparison will be unordered.

In [360]:
set(lowest20_indx) == set(top20_indx)

True

In [361]:
# Alternativley 
lowest20_indx.sort()
top20_indx.sort()
lowest20_indx == top20_indx

#test_data.iloc[top20_indx,:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool)

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [362]:
    '''
    Many ways to do it 
    1.Get margin or scores and predict which side of decision bounday each sample falls
    by looking at the sign (+ve/-ve) of the score for each data point
    
      scores = model.predict(data, output_type='margin')
      predicted_labels = [+1 if x > 0 else -1 for x in scores]
    
    2.Get probabilities that a given sample is a +ve setiment given sample and weights
    and if prob >= 0.5, assign +ve +1 sentiment, -ve -1 otherwise
    
      probs = model.predict(data, output_type='probability')
      predicted_labels = [+1 if x >= 0.5 else -1 for x in probs]
    
    3. Easiest - get prediction in form of class labels +1 or -1
    default output of model.predict is class predictions
    
      predicted_labels =  model.predict(data, output_type='class')  
      
    '''

"\nMany ways to do it \n1.Get margin or scores and predict which side of decision bounday each sample falls\nby looking at the sign (+ve/-ve) of the score for each data point\n\n  scores = model.predict(data, output_type='margin')\n  predicted_labels = [+1 if x > 0 else -1 for x in scores]\n\n2.Get probabilities that a given sample is a +ve setiment given sample and weights\nand if prob >= 0.5, assign +ve +1 sentiment, -ve -1 otherwise\n\n  probs = model.predict(data, output_type='probability')\n  predicted_labels = [+1 if x >= 0.5 else -1 for x in probs]\n\n3. Easiest - get prediction in form of class labels +1 or -1\ndefault output of model.predict is class predictions\n\n  predicted_labels =  model.predict(data, output_type='class')  \n  \n"

In [363]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predicted_labels =  model.predict(data) 
    
    # Compute the number of correctly classified examples
    # understand why this will always give num of data points regardless True/False
    # 1st to element-wise boolean comparision between the 2 arrays 
    # and then count how many comparisions returned True, i.e same class in both array !! 
    # but doesn't work - in fact (predicted_labels == true_labels) outputs 1s and 0s!!!!
    #print "num_correct using len()", len ( (predicted_labels == true_labels) == 'True' )
    print "num_correct using '(predicted_labels == true_labels).sum()' = ",\
    (predicted_labels == true_labels).sum() 
    #sum gives correct result as same as counting # of 1s, 1+0+1+0+0=2!!
    
    # np.equal(x1,x2) -- > Return (x1 == x2) element-wise
    # Parameters:	 x1, x2 : array_like : Input arrays of the same shape.
    # Returns:	  out : ndarray or bool :  Output array of bools, or a single bool if x1 and x2 are scalars.  
    
    num_correct = sum ( np.equal(predicted_labels, true_labels) )
    print "num_correct using 'sum ( np.equal(predicted_labels, true_labels) )' = ", num_correct
    print "num_data", data.shape[0]
    
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = num_correct * 1.0/data.shape[0]
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [364]:
print "Accuracy = %0.2f" %get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

num_correct using '(predicted_labels == true_labels).sum()' =  31078
num_correct using 'sum ( np.equal(predicted_labels, true_labels) )' =  31078
num_data 33336
Accuracy = 0.93


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). **0.91 from graphlab, 0.93 from sklearn**

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? **No**

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [365]:
dtype_dict = {'name':str, 'review':str, 'rating':int}
products = pd.read_csv('../../course-1/Week3/amazon_baby.csv', dtype = dtype_dict)

products = products.fillna({'review':''})  # fill in N/A's in the review column

def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
len(products) == len ( products[products['sentiment'] == -1]) + \
                len ( products[products['sentiment'] == 1])


True

In [366]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [367]:
len(significant_words)

for word in significant_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [368]:
products.head(2)

,name,review,rating,review_clean,sentiment,love,great,easy,old,little,...,broke,less,even,waste,disappointed,work,product,money,would,return
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [369]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.as_matrix()  #.to_numpy() if using SFrame
    label_sarray = data_sframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [370]:
#train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
#test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

# Use iloc for positional indexing

train_data = products.iloc[train_idx,:]
test_data = products.iloc[test_idx,:]
print len(train_data)  #133416
print len(test_data)   #33336

133416
33336


In [371]:
# Warning: This may take a few minutes...
feature_matrix_train, sentiment = get_numpy_data(train_data, significant_words, 'sentiment') 

/mnt/win-data/shared/stats-R/anaconda2/envs/conda_py2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [372]:
print feature_matrix_train.shape
feature_matrix_train

(133416, 21)


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Let's see what the first 5 samples in train dataset looks like:

In [373]:
train_data[0:5]['review']

0    These flannel wipes are OK, but in my opinion ...
1    it came early and was not disappointed. i love...
2    Very soft and comfortable and warmer than it l...
3    This is a product well worth the purchase.  I ...
4    All of my kids have cried non-stop when I trie...
Name: review, dtype: object

In [374]:
# To select multiple columns, simply pass a list of column names to the DataFrame, 
# the output of which will be a DataFrame.
print train_data[significant_words].head(5)

   love  great  easy  old  little  perfect  loves  well  able  car  broke  \
0     0      0     0    0       0        0      0     0     0    0      0   
1     1      0     0    0       0        0      0     0     0    0      0   
2     0      0     0    0       0        0      0     0     0    0      0   
3     2      0     0    0       1        0      1     1     0    0      0   
4     0      1     1    0       0        0      0     0     0    0      0   

   less  even  waste  disappointed  work  product  money  would  return  
0     0     0      0             0     0        0      0      0       0  
1     0     0      0             1     0        0      0      0       0  
2     0     0      0             0     0        0      0      0       0  
3     0     0      0             0     0        2      0      0       0  
4     0     0      0             0     1        0      0      0       0  


The **word_count** column in the graphlab version has been replaced by 20 columns here, each column has count for num of times that word appears in a given review - Inspect and you will see that the feature_matrix_train is similar to train_data[significant_words] with index series and col header removed.

In [375]:
feature_matrix_train[0:5,]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [376]:
# from sklearn.linear_model import LogisticRegression
# instantiate a logistic regression model, and fit with X and y
# X -> sparse matrix where each row is the word count vector for the corresponding review
simple_model = LogisticRegression()
X = feature_matrix_train
y = train_data['sentiment']
simple_model = model.fit(X, y)

# check the accuracy on the training set
simple_model.score(X, y)


0.79337560712358335

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [377]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predicted_labels =  model.predict(data) 
    num_correct = sum ( np.equal(predicted_labels, true_labels) )
    print "num_correct using 'sum ( np.equal(predicted_labels, true_labels) )' = ", num_correct
    print "num_data", data.shape[0]
    
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = num_correct * 1.0/data.shape[0]
    
    return accuracy

In [378]:
feature_matrix_test, sentiment = get_numpy_data(test_data, significant_words, 'sentiment') 
print "Accuracy = %0.2f" \
   %get_classification_accuracy(simple_model, feature_matrix_test, test_data['sentiment'])

num_correct using 'sum ( np.equal(predicted_labels, true_labels) )' =  26499
num_data 33336
Accuracy = 0.79


/mnt/win-data/shared/stats-R/anaconda2/envs/conda_py2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Now, we will inspect the weights (coefficients) of the **simple_model**:
Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [379]:
print simple_model.coef_
print "Non zero coefficients (including intercept) = " , \
np.count_nonzero(simple_model.coef_) + np.count_nonzero(simple_model.intercept_)

print "simple_model.coef_.shape:", simple_model.coef_.shape

print sort(simple_model.coef_)

[[ 0.46044587  1.02488403  0.62013718  0.80331622  0.11968815  0.26750161
   1.22672349  1.37217446  0.35096804  0.07555701  0.07830785 -1.41858188
  -0.20979965 -0.37127294 -1.61151138 -2.05293832 -0.59327438 -0.18733665
  -0.64718732 -0.36617285 -1.76055177]]
Non zero coefficients (including intercept) =  22
simple_model.coef_.shape: (1, 21)


NameError: name 'sort' is not defined

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [ ]:
print sum(1 for x in simple_model.coef_[0] if x > 0), \
     len([x for x in simple_model.coef_[0] if x > 0]) 

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [ ]:
positive_significant_words = simple_model.coefficients [simple_model.coefficients['value'] > 0 ]['index']
print positive_significant_words

simple_positive = simple_model.coefficients.sort('value', ascending=False)
simple_positive = simple_positive [simple_positive['value'] > 0 ]['index']

setimental_positive = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_positive = setimental_positive [setimental_positive['value'] > 0 ]['index']

print len(simple_positive)
print len(sentiment_positive)
simple_positive = simple_positive.to_numpy()
sentiment_positive = sentiment_positive.to_numpy()
print "These words have positive weights in both the models\n", np.intersect1d(simple_positive, sentiment_positive)

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [ ]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [ ]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? **sentiment_model**

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [ ]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [ ]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? **sentiment_model**

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [ ]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [ ]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative
print round(num_positive/len(test_data), 2)

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)? Better 91% with train data, but with test data accuracy was 87%. Baseline accuracy from majority class classfier is 84%. So we can't say that sentiment_model is definitely better than baseline, as its accuracy is only 3% better.